In [1]:
import gsc
import numpy as np
import re

# Just terminals are similar, agreement is probabilistic for SP

In [2]:
net = gsc.load_model('../training/s50Gram_nSim70_vSim00.pkl')

In [3]:
# .9 decay,.5 terminal, .5 phrase, 0.5 sentence 

np.random.seed(20181107)
input_decay_rate = 0.90

correctTrialTraces = []
attractionErrorTracesCollective = []
attractionErrorTracesOther = []
locErrorTraces = []


duration = 15
input_terminals = ['Ns:0/(1,1)','Np:1/(1,2)']
partial_input_terminals = ['Ns:1/(1,1)','Np:0/(1,2)']
input_phrases = ['NPCs[2]:0/(2,1)']
input_sentence = ['S[3]:0/(3,1)']
resultStrings = {}

curr_extC = np.zeros(net.num_bindings) ## start with zeros everywhere
idx = net.find_bindings(input_terminals)
curr_extC[idx] = 0.5
idx = net.find_bindings(partial_input_terminals)
curr_extC[idx] = 0.5
idx2 = net.find_bindings(input_phrases)
curr_extC[idx2] = 0.5
idx3 = net.find_bindings(input_sentence)
curr_extC[idx3] = 0.5

for trialcount in range(4000):

    net.reset(mu=net.ep, sd=0.02)
    
    net.extC += 1. * curr_extC # set conceptual level input
    net.ext = net.C2N(net.extC) # rotate to neural coordinates

    for i in range(duration):
        net.run(1)
        net.extC = net.extC * input_decay_rate
        net.ext = net.C2N(net.extC)

    
    #count up outputs
    output_elements = []
    for element in net.read_grid_point():
        f,r = element.split('/')
        if re.match(r'\(1\,',r) and f.find('@') == -1:
            fUnparsed,branch = f.split (':')
            output_elements.append(fUnparsed)  #strip locations
        if re.match(r'\(2\,',r) and f.find('@') == -1 and f.find('V') == -1:
            fUnparsed,branch = f.split (':')
            output_elements.append(fUnparsed)  #strip locations
    if (str(output_elements) in resultStrings):
        resultStrings[str(output_elements)] += 1
    else:
        resultStrings[str(output_elements)] = 1

    if (str(output_elements) == "['Ns', 'Np', 'Vs', 'NPCs[2]']"):
        correctTrialTraces.append(net.traces['actC'])
    if (str(output_elements) == "['Ns', 'Ns', 'Vs', 'NPCs[1]']"):
        locErrorTraces.append(net.traces['actC'])
    if (str(output_elements) == "['Ns', 'Np', 'Vp', 'NPCp[3]']"):
        attractionErrorTracesCollective.append(net.traces['actC'])
    ## otherwise, put it in the other one:
    if (str(output_elements) == "['Ns', 'Np', 'Vp', 'NPCs[1]']"):
        attractionErrorTracesOther.append(net.traces['actC'])
    if (str(output_elements) == "['Ns', 'Np', 'Vp', 'NPCs[2]']"):
        attractionErrorTracesOther.append(net.traces['actC'])
    if (str(output_elements) == "['Ns', 'Np', 'Vp', 'NPCp[1]']"):
        attractionErrorTracesOther.append(net.traces['actC'])
    if (str(output_elements) == "['Ns', 'Np', 'Vp', 'NPCp[2]']"):
        attractionErrorTracesOther.append(net.traces['actC'])
     
        
for key in resultStrings:
    print(key,resultStrings[key])    

['Ns', 'Np', 'NPCs[2]'] 548
['Ns', 'Np', 'Vs', 'NPCs[2]'] 1772
['Ns', 'Ns', 'Vs', 'NPCs[1]'] 683
['Ns', 'Np', 'Vp', 'NPCp[3]'] 760
['Ns', 'Vs', 'S[1]'] 59
['Ns', 'Ns', 'Vp', 'NPCs[1]'] 85
['Ns', 'Ns', 'NPCs[1]'] 48
['Np', 'Np', 'Vp', 'NPCp[2]'] 21
['Ns', 'Np', 'S[1]'] 3
['Ns', 'Np', 'Vp', 'NPCs[2]'] 4
['Ns', 'Np', 'Vs', 'Vs', 'NPCs[2]'] 1
['Ns', 'Np', 'Vp', 'Vs', '*Ns', 'RC[2]'] 6
['Np', 'Ns', 'Vp', 'NPCp[1]'] 7
['Ns', 'Np', 'Vp', 'Vp', '*Ns', 'RC[2]'] 1
['Np', 'Ns', 'NPCp[1]'] 1
['Ns', 'Np', 'Vp', 'NPCs[2]', 'RC[2]'] 1


In [4]:
np.save('correctTrialTraces-Best.npy',correctTrialTraces)
np.save('attractionErrorTracesCollective-Best.npy',attractionErrorTracesCollective)
np.save('attractionErrorTracesOther-Best.npy',attractionErrorTracesOther)
np.save('locErrorTraces-Best.npy',locErrorTraces)
np.save('resultStrings-Best.npy',resultStrings)